![logo](./img/LogoLine_horizon_C3S.png)

## Tutorial on fetching CARRA data from CDS

CARRA data can be found in the CDS (Copernicus Climate Data Store, https://cds.climate.copernicus.eu/cdsapp#!/home).
You need to create an account and accept the terms and conditions for all datasets that you want to use.


### Fetching CARRA data
The source coode will download the month of June 2023. By specifying the start and end date below you can easely download longer periods.
Here, we download the 3-hourly analysis time steps and the data will be stored in a specified file.

---

In [4]:
import cdsapi
import os

DATADIR="CARRA"
if not os.path.isdir(DATADIR):
    os.makedirs(os.path.join(DATADIR,"Raw_data"))
c = cdsapi.Client()

def fetch_one_month(month, year):
    c.retrieve(
        'reanalysis-carra-single-levels',
        {
            'format': 'grib',
            'domain': 'west_domain',
            'level_type': 'surface_or_atmosphere',
            'variable': '2m_temperature',
            'product_type': 'analysis',
            'year': year,
            'month': month,
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '03:00', '06:00',
                '09:00', '12:00', '15:00',
                '18:00', '21:00',
            ],
        },
        os.path.join(DATADIR,"Raw_data",'T2m_an_'+str(year)+"{:02d}".format(month)+'.grb'))

    
yearStart = 2023
yearEnd   = 2023
monthStart =  6
monthEnd   =  6
for year in list(range(yearStart, yearEnd + 1)):
    for month in list(range(1,13)):
        if year == yearStart and month < monthStart:
            # Before the start date.
            continue
        if year == yearEnd and month > monthEnd:
            # After end date.
            break

        print ("Data will be fetched for ", year, month)
        fetch_one_month(month, year)

Data will be fetched for  2023 6


2024-05-15 12:22:12,442 INFO Welcome to the CDS
2024-05-15 12:22:12,443 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-carra-single-levels
2024-05-15 12:22:12,546 INFO Request is completed
2024-05-15 12:22:12,547 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data5/adaptor.mars.external-1715768399.6479416-7306-18-2fd9f25c-da29-4e23-ac48-2b6b97793827.grib to CARRA/Raw_data/T2m_an_202306.grb (621.2M)
2024-05-15 12:24:47,014 INFO Download rate 4M/s                                 


### Fetching ERA5 data

Here, you downlaod one month of ERA5 data (June 2023). Same as for CARRA, we download 3-hourly data though ERA5 does offer hourly analyses.

---

In [5]:
DATADIR=os.path.join("CARRA","ERA5")
if not os.path.isdir(DATADIR):
    os.makedirs(DATADIR)
                
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': '2m_temperature',
        'year': '2023',
        'month': '06',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
        ],
        'time': [
            '00:00', '03:00', '06:00',
            '09:00', '12:00', '15:00',
            '18:00', '21:00',
        ],
        'area': [
            90, -110, 55,
            40,
        ],
        'format': 'netcdf',
    },
    os.path.join(DATADIR,'era5_t2m_202306.nc'))

2024-05-15 12:27:20,900 INFO Welcome to the CDS
2024-05-15 12:27:20,901 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-05-15 12:27:21,525 INFO Request is queued
2024-05-15 12:27:22,593 INFO Request is running
2024-05-15 12:28:11,317 INFO Request is completed
2024-05-15 12:28:11,318 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.mars.internal-1715768883.9647436-16931-13-032aa7aa-fae9-49f8-845a-43b7d22902f6.nc to CARRA/ERA5/era5_t2m_202306.nc (38.8M)
2024-05-15 12:28:17,870 INFO Download rate 5.9M/s                               


Result(content_length=40680676,content_type=application/x-netcdf,location=https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data3/adaptor.mars.internal-1715768883.9647436-16931-13-032aa7aa-fae9-49f8-845a-43b7d22902f6.nc)